In [1]:
import numpy as onp
import tensorflow as tf
import matplotlib.pyplot as plt

from utils import *

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPU


# hyperparameters

In [3]:
#data
DATASET = 'cifar10'
class_num   = 10
test_size   = 2048
image_shape = None

if DATASET =='mnist':
    image_shape = (28, 28)
elif DATASET == 'cifar10':
    image_shape = (32, 32, 3)

#training
run = 10
batch_size = 256
epoch = 100

In [4]:
x_train_all, y_train_all, x_test_all, y_test_all = tuple(onp.array(x) for x in get_dataset(DATASET, None, None, 
                                                                                  do_flatten_and_normalize=False))

In [5]:
FGSM = onp.load('./cifar-FGSM-eps=0.03.npy')
I_FGSM = onp.load('./cifar-IFGSM-eps=0.03.npy')

In [6]:
y_test = onp.load("./npy/%s_y_test.npy"%(DATASET))

In [7]:
x_test = onp.load("./npy/%s_x_test.npy"%(DATASET))

In [8]:
# class Model(tf.keras.Model):
#     def __init__(self, layers):
#         super(Model, self).__init__()
#         self.layers_num = layers
#         self.conv = [tf.keras.layers.Conv2D(64, (3,3)) for i in range(self.layers_num)]
#         self.flatten = tf.keras.layers.Flatten()
#         self.out = tf.keras.layers.Dense(10)
        
#     def call(self, inputs):
#         x = inputs
#         for f in self.conv:
#             x = f(x)
#             x = tf.math.erf(x)
            
#         x = self.flatten(x)
#         out = self.out(x)
#         return out

In [9]:
layers = tf.keras.layers

In [10]:
img_input = layers.Input(shape=image_shape)
x = layers.Conv2D(64,  (3, 3), activation='relu', padding='same')(img_input)
x = layers.Conv2D(64,  (3, 3), activation='relu', padding='same')(x)
x = 
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPool2D()(x)
x = layers.Flatten()(x)
x = layers.Dense(4096, activation='relu')(x)
x = layers.Dense(1000, activation='relu')(x)
x = layers.Dense(1000, activation='relu')(x)
out = layers.Dense(10)(x)

model = tf.keras.Model(inputs=img_input, outputs=out)

In [11]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2, momentum=0.9, nesterov=True),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [14]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 128)        

In [12]:
x_train_all, x_test_all = x_train_all.reshape(-1, 32, 32, 3), x_test_all.reshape(-1, 32, 32, 3)

In [13]:
model.fit(x_train_all, y_train_all, batch_size=batch_size, epochs=20, validation_split=0.1)

Epoch 1/20
176/176 [==============================] - 16s 92ms/step - loss: 2.3027 - accuracy: 0.0980 - val_loss: 2.3030 - val_accuracy: 0.0948
Epoch 2/20
176/176 [==============================] - 8s 45ms/step - loss: 2.3027 - accuracy: 0.0985 - val_loss: 2.3030 - val_accuracy: 0.0940
Epoch 3/20
176/176 [==============================] - 11s 62ms/step - loss: 2.3027 - accuracy: 0.0982 - val_loss: 2.3029 - val_accuracy: 0.0948
Epoch 4/20
176/176 [==============================] - 7s 42ms/step - loss: 2.3027 - accuracy: 0.0979 - val_loss: 2.3028 - val_accuracy: 0.0940
Epoch 5/20
176/176 [==============================] - 7s 42ms/step - loss: 2.3027 - accuracy: 0.0979 - val_loss: 2.3028 - val_accuracy: 0.0994
Epoch 6/20
176/176 [==============================] - 7s 42ms/step - loss: 2.3027 - accuracy: 0.0987 - val_loss: 2.3029 - val_accuracy: 0.0940
Epoch 7/20
176/176 [==============================] - 7s 42ms/step - loss: 2.3027 - accuracy: 0.1001 - val_loss: 2.3029 - val_accuracy: 0.09

In [12]:
model.evaluate(x_test.reshape(-1, 32, 32, 3), y_test, verbose=0)

[2.493234734982252, 0.46142578]

In [ ]:
model.evaluate(FGSM[2].reshape(-1, 32, 32, 3), y_test, verbose=0)

In [ ]:
loss_curve = []
acc_curve  = []

loss_curve_FGSM = {idx*10+1: [] for idx in range(len(FGSM))}
acc_curve_FGSM  = {idx*10+1: [] for idx in range(len(FGSM))}

loss_curve_PGD_10 = {idx*10+1: [] for idx in range(len(FGSM))}
acc_curve_PGD_10  = {idx*10+1: [] for idx in range(len(FGSM))}

loss_curve_PGD_100 = {idx*10+1: [] for idx in range(len(FGSM))}
acc_curve_PGD_100  = {idx*10+1: [] for idx in range(len(FGSM))}

for i in range(run):
    model = Model(10)
    model.compile(optimizer='adam',
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    
    for j in range(int(epoch)):
        model.fit(x_train_all, y_train_all, batch_size=batch_size, epochs=10)
        # CLEAN
        r = model.evaluate(x_test, y_test, verbose=0)
        loss_curve.append(r[0])
        acc_curve.append(r[1])
        
        for idx, (F, _10, _100) in enumerate(zip(FGSM, PGD_10, PGD_100)):
                
            r = model.evaluate(F,    y_test, verbose=0)
            print("FGSM    %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
            loss_curve_FGSM[idx*10+1].append(r[0])
            acc_curve_FGSM[idx*10+1].append(r[1])
        
            r = model.evaluate(_10,  y_test, verbose=0)
            print("PGD-10  %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
            loss_curve_PGD_10[idx*10+1].append(r[0])
            acc_curve_PGD_10[idx*10+1].append(r[1])
            
            r = model.evaluate(_100, y_test, verbose=0)
            print("PGD-100 %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
            loss_curve_PGD_100[idx*10+1].append(r[0])
            acc_curve_PGD_100[idx*10+1].append(r[1])
            

In [ ]:
loss_curve_FGSM

In [ ]:
for idx, (F, _10, _100) in enumerate(zip(FGSM, PGD_10, PGD_100)):
    r = model.evaluate(F,    y_test, verbose=0)
    print("FGSM    %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
    r = model.evaluate(_10,  y_test, verbose=0)
    print("PGD-10  %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))
    r = model.evaluate(_100, y_test, verbose=0)
    print("PGD-100 %d,  loss: %.3f, acc: %.3f"%(idx*10+1, r[0], r[1]))